In [1]:
import numpy as np
import cv2

In [3]:
img = cv2.imread('inveraraycastle.jpg') 
gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
surf = cv2.xfeatures2d.SIFT_create()

error: OpenCV(4.0.1) C:\ci\opencv-suite_1573470242804\work\opencv_contrib-4.0.1\modules\xfeatures2d\src\sift.cpp:1207: error: (-213:The function/feature is not implemented) This algorithm is patented and is excluded in this configuration; Set OPENCV_ENABLE_NONFREE CMake option and rebuild the library in function 'cv::xfeatures2d::SIFT::create'


In [1]:
import numpy as np 
import cv2 
from matplotlib import pyplot as plt
img = cv2.imread('arithmetic.jpg', 0)
# Initiate FAST object with default values 
fast = cv2.FastFeatureDetector_create()
# find and draw the keypoints 
kp = fast.detect(img, None) 
img2 = cv2.drawKeypoints(img, kp, None, color=(255,0,0))
# Print all default params 
print("Threshold: {}".format(fast.getThreshold())) 
print("nonmaxSuppression:{}".format(fast.getNonmaxSuppression())) 
print("neighborhood: {}".format(fast.getType())) 
print("Total Keypoints with nonmaxSuppression: {}".format(len(kp)))
cv2.imwrite('ShiTomasiBlock_fast_true.png', img2)
cv2.imshow('ShiTomasiBlock_fast_true', img2)
cv2.waitKey(0)


Threshold: 10
nonmaxSuppression:True
neighborhood: 2
Total Keypoints with nonmaxSuppression: 3013


32

In [2]:
# Disable nonmaxSuppression 
fast.setNonmaxSuppression(0)
img = cv2.imread('arithmetic.jpg', 0)
kp = fast.detect(img, None)
# print("Total Keypoints without nonmaxSuppression: {}“ \ .format(len(kp))) 
img3 = cv2.drawKeypoints(img, kp, None, color=(255,0,0))
cv2.imwrite('ShiTomasiBlock_fast_false.png', img3)
cv2.imshow('ShiTomasiBlock_fast_false', img3)
cv2.waitkey(0)

NameError: name 'fast' is not defined

In [2]:
3%7

3

In [5]:
import numpy as np 
import cv2 
from matplotlib import pyplot as plt
# queryImage 
img1 = cv2.imread('inveraraycastle.jpg', cv2.IMREAD_GRAYSCALE) 
# trainImage 
img2 = cv2.imread('inveraraycastle-2.jpg', cv2.IMREAD_GRAYSCALE)
# Initiate ORB detector
orb = cv2.ORB_create()
# find the keypoints and descriptors with ORB 
kp1, des1 = orb.detectAndCompute(img1, None) 
kp2, des2 = orb.detectAndCompute(img2, None)
# create BFMatcher object 
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# Match descriptors. 
matches = bf.match(des1, des2)
# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)
# Draw first 10 matches.
img3 = cv2.drawMatches(img1, kp1, img2, kp2, matches[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#plt.imshow(img3), plt.show() 
cv2.imwrite('book_in_scene_BfMatch.jpg', img3) 
cv2.imshow('book_in_scene_BfMatch', img3)
cv2.waitKey(0)

-1

In [13]:
import numpy as np 
import cv2 
from matplotlib import pyplot as plt
# queryImage 
img1 = cv2.imread('inveraraycastle.jpg', cv2.IMREAD_GRAYSCALE) 
# trainImage 
img2 = cv2.imread('inveraraycastle-2.jpg', cv2.IMREAD_GRAYSCALE)

orb = cv2.ORB_create()
# find the keypoints and descriptors with ORB 
kp1, des1 = orb.detectAndCompute(img1, None) 
kp2, des2 = orb.detectAndCompute(img2, None)
print(des1, des2)
# FLANN parameters 
FLANN_INDEX_LSH = 1 
index_params              = dict(algorithm = FLANN_INDEX_LSH,
                   table_number = 6,
                   key_size = 12,
                   multi_probe_level = 1)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)
# store all the good matches as per Lowe's ratio test. 
good = [] 
for m,n in matches: 
    if m.distance < 0.7*n.distance:
        good.append(m)
if len(good)>MIN_MATCH_COUNT: 
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp1[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matchesMask = mask.ravel().tolist()
    h,w = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)
    img2 = cv2.polylines(img2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)
else:
    print("Not enough matches are found - %d/%d" % (len(good), MIN_MATCH_COUNT))
    matchesMask = None
draw_params = dict(matchColor = (0,255,0), 
                   singlePointColor = None, 
                   matchesMask = matchesMask,
                   flags = 2)
img3 = cv2.drawMatches(img1, kp1, img2, kp2, good, None, **draw_params)
cv2.imshow('book_in_scene_homography', img3)
cv2.imwrite('Output.jpg', img3)
cv2.waitKey(0)

[[ 79 168 228 ... 141  92 238]
 [  2 127  51 ...  29  21 221]
 [ 68  76 158 ... 243 165   3]
 ...
 [189 205  69 ... 159 216 212]
 [ 79 177 133 ...  39 177 117]
 [ 62 156 237 ... 126 214 251]] [[ 84 131  95 ... 139   1  33]
 [  2 143  40 ...   4 141 251]
 [143 175  66 ... 247  43 197]
 ...
 [242 219 110 ... 132 180  91]
 [  5 142 211 ... 107 129  32]
 [ 10 123 166 ... 244 162 251]]


error: OpenCV(4.0.1) C:\ci\opencv-suite_1573470242804\work\modules\flann\src\miniflann.cpp:315: error: (-210:Unsupported format or combination of formats) in function 'cv::flann::buildIndex_'
> type=0
> 